In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [53]:
rating = pd.read_csv ("/content/rating_tva.csv")
consolidated = pd.read_csv ("/content/consolidated_tva.csv")
movie = pd.read_csv ("/content/movie_tva.csv")

<ipython-input-53-be6e935b32a6>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  rating = pd.read_csv ("/content/rating_tva.csv")


In [3]:
rating.head()

,userId,movieId,rating,timestamp
0,1.0,2.0,3.5,4/2/2005 23:53
1,1.0,29.0,3.5,4/2/2005 23:31
2,1.0,32.0,3.5,4/2/2005 23:33
3,1.0,47.0,3.5,4/2/2005 23:32
4,1.0,50.0,3.5,4/2/2005 23:29


In [37]:
consolidated.head()

,movieId,Title,Rating,Count of Reviews,Adventure,Animation,Children,Comedy,Fantasy,Horror,...,Drama,War,Musical,Imax,Documentary,Crime,Film-Noir,Western,Action,No genre
0,1,Toy Story (1995),3.97,241,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),3.25,112,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),3.23,63,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),3.39,13,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),3.25,69,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [38]:
consolidated.columns
consolidated_copy = consolidated

In [109]:
consolidated_copy = consolidated_copy.loc[consolidated_copy['Count of Reviews'] != 'No Reviews']
consolidated_copy = consolidated_copy.loc[consolidated_copy['Rating'] != 'No Rating'] 
consolidated_copy = consolidated_copy.loc[consolidated_copy['No genre'] != 1] 
consolidated_copy = consolidated_copy.drop('No genre', axis=1)

# General

In [111]:
consolidated_copy['Rating'] = consolidated_copy['Rating'].astype(float)
c = consolidated_copy['Rating'].mean()
c

3.2817977391515742

In [112]:
consolidated_copy['Count of Reviews'] = consolidated_copy['Count of Reviews'].astype(float)
m= consolidated_copy['Count of Reviews'].quantile(0.9)
m

30.0

In [113]:
mostviewed_movies = consolidated_copy.copy().loc[consolidated_copy['Count of Reviews'] >= m]
mostviewed_movies.shape

(846, 23)

In [114]:
def weighted_rating(x, m=m, C=c):
    v = x['Count of Reviews']
    R = x['Rating']

    return (v/(v+m) * R) + (m/(m+v) * C)

In [115]:
mostviewed_movies['score'] = mostviewed_movies.apply(weighted_rating, axis=1)

In [116]:
#Sort movies based on score calculated above
mostviewed_movies = mostviewed_movies.sort_values('score', ascending=False)

#Print the top 15 movies
mostviewed_movies[['Title', 'Count of Reviews', 'Rating', 'score']].head(15)

,Title,Count of Reviews,Rating,score
315,"Shawshank Redemption, The (1994)",305.0,4.41,4.308967
843,"Godfather, The (1972)",197.0,4.40,4.252220
1195,"Godfather: Part II, The (1974)",137.0,4.42,4.215533
49,"Usual Suspects, The (1995)",224.0,4.34,4.215015
1169,One Flew Over the Cuckoo's Nest (1975),148.0,4.36,4.178281
523,Schindler's List (1993),247.0,4.26,4.154058
2873,Fight Club (1999),207.0,4.27,4.144911
887,Rear Window (1954),79.0,4.40,4.092238
4877,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",128.0,4.27,4.082367
4132,Memento (2000),161.0,4.21,4.064209


# Cosine Similartity

In [117]:
tfidf = TfidfVectorizer(stop_words='english')
movie['genres'] = movie['genres'].str.replace('[^\w\s]','')
movie['genres'] = movie['genres'].fillna('')


tfidf_matrix = tfidf.fit_transform(movie['genres'])
tfidf_matrix.shape


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(movie.index, index=movie['title']).drop_duplicates()

<ipython-input-117-6765c6dfb63b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['genres'] = movie['genres'].str.replace('[^\w\s]','')


In [118]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    movie_indices = [i[0] for i in sim_scores]
    return movie['title'].iloc[movie_indices]

In [132]:
get_recommendations('Orphans (1997)')

74                               Big Bully (1996)
81                Antonia's Line (Antonia) (1995)
83             Last Summer in the Hamptons (1995)
94                  In the Bleak Midwinter (1995)
104    Nobody Loves Me (Keiner liebt mich) (1994)
131                              Nueba Yol (1995)
154                       Blue in the Face (1995)
169                                Jeffrey (1995)
176                   Love & Human Remains (1993)
192                                  Smoke (1995)
Name: title, dtype: object

# KNN

In [120]:
from scipy.sparse import csr_matrix
movies_users=rating.pivot_table(index='movieId', columns='userId',values='rating',aggfunc='mean').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)

In [121]:
from sklearn.neighbors import NearestNeighbors
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=15)
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=15)

In [122]:
!pip install fuzzywuzzy
from fuzzywuzzy import process

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [123]:
def recommender(movie_name, data,model, n_recommendations ):
  model.fit(data)
  idx=process.extractOne(movie_name, movie['title'])[2]
  print('Movie Selected: ',movie['title'][idx], 'Index: ',idx)
  distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
  for i in indices:
    print(movie['title'][i].where(i!=idx))
    

In [136]:
recommender('Hansel & Gretel (2002)', mat_movies_users, model_knn,20)

Movie Selected:  Hansel & Gretel (2002) Index:  5584
7690                     Anne of the Thousand Days (1969)
6071                           Black Stallion, The (1979)
4891    Spacehunter: Adventures in the Forbidden Zone ...
6069                                10 to Midnight (1983)
4892                            White Water Summer (1987)
5396                                 In Like Flint (1967)
5709                          Weight of Water, The (2000)
5650                                   Ghost Story (1981)
4251                             Bride of the Wind (2001)
6313                                    Shenandoah (1965)
5631              Enter the Ninja (a.k.a. Ninja I) (1981)
4956                     Time of Favor (Ha-Hesder) (2000)
4962                Heart of Glass (Herz aus Glas) (1976)
6296         White Sheik, The (Sceicco bianco, Lo) (1952)
4976                                    Metropolis (2001)
6058                                     Daredevil (2003)
5584               